In [1]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import torchvision

In [1]:
train_path = '/home/kaumad/roco-dataset/train'
train_img_dir = os.path.join(train_path, 'radiology', 'images')
val_path = '/home/kaumad/roco-dataset/validation'
val_img_dir = os.path.join(val_path, 'radiology', 'images')
test_path = '/home/kaumad/roco-dataset/test'
test_img_dir = os.path.join(test_path, 'radiology', 'images')

In [41]:
import torchvision

def read_lines_old(img_dir, csv_file):
    lines = []
    for idx, row in csv_file.iterrows():
        img_path = os.path.join(img_dir, row['name'])
        if not os.path.exists(img_path):
            continue
        try:
            data = torchvision.io.image.read_file(img_path)
        except:
            print(img_path)
            continue
        # TODO check if the image is not empty
        caption = row['caption'].lower().rstrip().replace("\\n", "").rstrip(".")
        # caption = caption.replace("\\n", "").rstrip()
        # print([caption])
        caption = f"{caption}"
        line = json.dumps({"image_path": img_path, "captions": [caption]})
        lines.append(line)

        # if id>5:
        #     break
    return lines


In [3]:
def has_unicode(text):
    for c in text:
        if ord(c)>127:
            print(f"unicode: {c}", text)
            return True
    return False

In [4]:
import collections

lens = []

def read_lines(img_dir, csv_file, n_lines=0):
    lines = []
    image_path_to_caption = collections.defaultdict(list)
    for idx, row in csv_file.iterrows():
        if n_lines > 0 and idx > n_lines:
            break
        img_path = os.path.join(img_dir, row['name'])
        if not os.path.exists(img_path):
            continue
        try:
            data = torchvision.io.image.read_file(img_path)
        except:
            print(img_path)
            continue
        # TODO check if the image is not empty
        caption = row['caption'].lower().rstrip().replace(
            "\\n", "").rstrip(".")
        try:
            caption = caption.encode('ascii')
        except:
            continue
        
        lens.append(len(caption))
        
        if len(caption) < 10:
            continue
        # caption = caption.replace("\\n", "").rstrip()
        # print([caption])
        # has_unicode(caption)
        caption = f"{caption}"
        image_path_to_caption[img_path].append(caption)

    for img_path, captions in image_path_to_caption.items():
        line = json.dumps({"image_path": img_path, "captions": captions})
        lines.append(line)

    return lines


In [7]:
train_csv = pd.read_csv(os.path.join(train_path, 'radiology', 'traindata.csv'))
lines = read_lines(train_img_dir, train_csv, n_lines=0)

/home/kaumad/roco-dataset/train/radiology/images/PMC4083729_AMHSR-4-14-g002.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC2837471_IJD2009-150251.001.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC2505281_11999_2007_30_Fig6_HTML.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC3745845_IJD2013-683423.005.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC4917066_amjcaserep-17-301-g001.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC4805615_13244_2016_481_Fig12_HTML.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC2584650_1757-1626-1-193-1.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC3283944_JISP-15-414-g006.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC4946383_HI-10-1-25-g003.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC5646151_TOORTHJ-11-882_F2.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC2505289_11999_2007_20_Fig1_HTML.jpg
/home/kaumad/roco-dataset/train/radiology/images/PMC3932583_IJRI-23-379-g009

In [56]:
import numpy as np
lens = np.array(lens)

In [46]:
json_dir = '../../../data'
with open(os.path.join(json_dir, "train_dataset.json"), "w") as f:
    f.write("\n".join(lines))

In [48]:
val_csv = pd.read_csv(os.path.join(val_path, 'radiology', 'valdata.csv'))
lines = read_lines(val_img_dir, val_csv, n_lines=0)

In [49]:
with open(os.path.join(json_dir, "valid_dataset-sample.json"), "w") as f:
    f.write("\n".join(lines))

In [17]:
# Used only for creating sample dataset
train_lines = lines[:45000]
val_lines = lines[45000:]

json_dir = '../../../data'
with open(os.path.join(json_dir, "train_dataset.json"), "w") as f:
    f.write("\n".join(train_lines))

with open(os.path.join(json_dir, "valid_dataset.json"), "w") as f:
    f.write("\n".join(val_lines))

In [ ]:
caption = " axial computed tomography scan of the pelvis showing a diffuse infiltration of the bladder wall, catheter in situ (arrow).\\n"
caption.rstrip("\\n")